In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
experiment_relative_path = r"outputs\2025-08-06_09-53-26_eda_shear_50M"
test_data_path = r"C:\Users\cervinka\cervinka\GitHub\MathCAS\datasets\dataset_compressible_flow_5M_test.csv"

In [3]:
# Load model architecture and weights from experiment directory #

import torch
from pathlib import Path
from config import load_config
from src.architecture import NeuralNetwork

exp_dir = project_root / Path(experiment_relative_path)

# Now use exp_dir for all further paths
config_path = exp_dir / "config.yaml"
model_path = exp_dir / "best_model.pt"

config = load_config(str(config_path))

# Reconstruct model architecture
model = NeuralNetwork(config.architecture)

# Load trained weights
state_dict = torch.load(model_path, map_location="cpu", weights_only=True)
model.load_state_dict(state_dict)

print(model)

NeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=9, out_features=250, bias=True)
    (1): ReLU()
    (2): Linear(in_features=250, out_features=250, bias=True)
    (3): ReLU()
    (4): Linear(in_features=250, out_features=250, bias=True)
    (5): ReLU()
    (6): Linear(in_features=250, out_features=250, bias=True)
    (7): ReLU()
    (8): Linear(in_features=250, out_features=250, bias=True)
    (9): ReLU()
    (10): Linear(in_features=250, out_features=250, bias=True)
    (11): ReLU()
    (12): Linear(in_features=250, out_features=250, bias=True)
    (13): ReLU()
    (14): Linear(in_features=250, out_features=3, bias=True)
  )
)


In [5]:
import pandas as pd
import torch
import time
from torch.utils.data import DataLoader, TensorDataset
from utils.metrics import compute_regression_metrics  

# Load test data
test_df = pd.read_csv(test_data_path, usecols=config.data.in_cols + config.data.out_cols)
X_test = torch.tensor(test_df[config.data.in_cols].values, dtype=torch.float32)
y_test = torch.tensor(test_df[config.data.out_cols].values, dtype=torch.float32)

# Create DataLoader for test set
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=config.data.batch_size or 128, shuffle=False)

# Set model to eval mode
model.eval()
all_preds, all_targets = [], []

start_time = time.time()
with torch.no_grad():
    for batch_x, batch_y in test_loader:
        preds = model(batch_x)
        all_preds.append(preds.cpu())
        all_targets.append(batch_y.cpu())
inference_time = time.time() - start_time

y_pred = torch.cat(all_preds, dim=0).numpy()
y_true = torch.cat(all_targets, dim=0).numpy()

metrics = compute_regression_metrics(y_true, y_pred)
# Print metrics for each output column separately
output_cols = config.data.out_cols  # list of output column names

for i, col in enumerate(output_cols):
    y_true_col = y_true[:, i]
    y_pred_col = y_pred[:, i]
    metrics = compute_regression_metrics(y_true_col, y_pred_col)
    print(f"\nMetrics for output '{col}':")
    for k, v in metrics.items():
        print(f"  {k}: {v}")


Metrics for output 'ResVort':
  MAPE: 5.4520e+01
  MAE: 4.3453e-03
  MSE: 1.0271e-04
  MedAE: 9.4187e-04
  MaxAE: 2.8947e-01

Metrics for output 'ResStrain':
  MAPE: 2.3260e+01
  MAE: 6.9658e-03
  MSE: 1.9044e-04
  MedAE: 4.5735e-03
  MaxAE: 5.3292e-01

Metrics for output 'Shear':
  MAPE: 3.7261e-01
  MAE: 4.5275e-03
  MSE: 7.0177e-05
  MedAE: 2.8708e-03
  MaxAE: 2.6604e-01


In [6]:
import numpy as np

for i, col in enumerate(config.data.out_cols):
    print(f"\ntrue, prediction for output '{col}':")
    indices = np.random.choice(len(y_true), size=10, replace=False)
    for idx in indices:
        true_val = np.ravel(y_true[idx])[i]
        pred_val = np.ravel(y_pred[idx])[i]
        print(f"{true_val:.5e}, {pred_val:.5e}")


true, prediction for output 'ResVort':
0.00000e+00, -1.37329e-04
6.93117e-01, 6.99159e-01
0.00000e+00, -2.19731e-04
9.83157e-03, 8.81432e-03
0.00000e+00, 2.90776e-04
0.00000e+00, -2.63471e-05
6.08158e-01, 6.11386e-01
0.00000e+00, -5.76600e-05
1.12723e-01, 1.14564e-01
0.00000e+00, -4.26030e-04

true, prediction for output 'ResStrain':
3.14290e-01, 3.24006e-01
1.57298e+00, 1.57794e+00
4.39599e-01, 4.37786e-01
4.92036e-01, 4.72888e-01
1.20406e+00, 1.21047e+00
4.16279e-01, 4.10967e-01
9.84596e-04, 4.60233e-03
9.55960e-01, 9.54306e-01
6.57656e-01, 6.53229e-01
1.32741e+00, 1.31725e+00

true, prediction for output 'Shear':
9.10774e-01, 9.09212e-01
1.42458e+00, 1.42542e+00
1.48472e+00, 1.48328e+00
1.63921e+00, 1.63194e+00
1.00771e+00, 1.00517e+00
1.63139e+00, 1.50591e+00
1.81977e+00, 1.82615e+00
1.68361e+00, 1.68297e+00
3.67199e-01, 3.63743e-01
1.68948e+00, 1.69804e+00


In [8]:
# Save metrics to a txt file #
import numpy as np

results_lines = []

output_cols = config.data.out_cols
for i, col in enumerate(output_cols):
    y_true_col = y_true[:, i]
    y_pred_col = y_pred[:, i]
    metrics = compute_regression_metrics(y_true_col, y_pred_col)
    results_lines.append(f"\nMetrics for output '{col}':")
    for k, v in metrics.items():
        results_lines.append(f"  {k}: {v}")

# Save 10 random predictions for each output
for i, col in enumerate(output_cols):
    results_lines.append(f"\ntrue, prediction for output '{col}':")
    indices = np.random.choice(len(y_true), size=10, replace=False)
    for idx in indices:
        true_val = np.ravel(y_true[idx])[i]
        pred_val = np.ravel(y_pred[idx])[i]
        results_lines.append(f"{true_val:.5e}, {pred_val:.5e}")

# Write to file in experiment folder
results_path = exp_dir / "test_results.txt"
with open(results_path, "w") as f:
    for line in results_lines:
        f.write(line + "\n")

print(f"Results saved to {results_path}")

Results saved to c:\Users\cervinka\cervinka\GitHub\MathCAS\outputs\2025-08-04_22-52-18_dan_250\test_results.txt
